# LangGraph with AgentCore Memory Tool (短期記憶)

## はじめに
このノートブックでは、LangGraphフレームワークを使用して、Amazon Bedrock AgentCoreのメモリ機能を会話型AIエージェントと統合する方法を説明します。**短期記憶**の保持にフォーカスし、単一の会話セッション内でエージェントが明示的なコンテキスト管理なしに、会話の前の部分から情報を思い出せるようにします。


## チュートリアルの詳細

| 項目                | 詳細                                                                             |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ | 短期会話型                                                                       |
| エージェント用途     | パーソナルフィットネス                                                           |
| エージェントフレームワーク | Langgraph                                                                        |
| LLMモデル           | Anthropic Claude Sonnet 3                                                        |
| チュートリアル構成要素 | AgentCore短期メモリ、Langgraph、ツールによるメモリ検索                            |
| 例の複雑度          | 初級                                                                             |

学習内容：
- AgentCoreメモリを使用した短期記憶用メモリストアの作成
- LangGraphを使用した構造化メモリワークフローを持つエージェントの作成
- 会話履歴検索用メモリツールの実装
- 単一セッション内でのコンテキスト情報へのアクセスと利用
- 効果的なメモリ想起による会話体験の向上


### シナリオのコンテキスト

この例では、会話を通して言及されるワークアウトの詳細、フィットネス目標、身体的制限、運動の好みを記憶できる「**パーソナルフィットネスコーチ**」を作成します。このアシスタントは、効果的な短期記憶管理により、ユーザーが情報を繰り返し述べる必要なく、より自然でパーソナライズされたフィットネスコーチング体験を可能にする方法を実演します。


## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件

- Python 3.10+
- 適切な権限を持つAWSアカウント
- AgentCoreメモリに適切な権限を持つAWS IAMロール
- Amazon Bedrockモデルへのアクセス

環境をセットアップして始めましょう！

## ステップ1: 環境セットアップ
このノートブックを動作させるために必要なライブラリをすべてインポートし、クライアントを定義しましょう。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime

Amazon BedrockモデルとAgentCoreに適切な権限を持つリージョンとロールを定義します

In [ ]:
import os
region = os.getenv('AWS_REGION', 'us-west-2')

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

### 統合の仕組み

LangGraphとAgentCoreメモリの統合には以下が含まれます：

1. AgentCoreメモリを使用して会話を短期記憶に保存する
2. LangGraphの構造化ワークフローでメモリ操作を管理する

このアプローチにより、メモリ管理を推論から分離し、よりクリーンで保守しやすいエージェントアーキテクチャを作成できます。

## ステップ2: メモリ作成
このセクションでは、AgentCoreメモリSDKを使用してメモリストアを作成します。このメモリストアにより、エージェントが会話からの情報を保持できるようになります。

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from botocore.exceptions import ClientError

In [ ]:
client = MemoryClient(region_name=region)
memory_name = "FitnessCoach"
memory_id = None

In [ ]:
try:
    print("メモリを作成中...")
    # メモリリソースを作成
    memory = client.create_memory_and_wait(
        name=memory_name,                       # このアカウント内のすべてのメモリで一意の名前
        description="フィットネスコーチエージェント",      # 人間が読める説明
        strategies=[],                          # 短期記憶にはメモリストラテジーなし
        event_expiry_days=7,                    # メモリは7日後に期限切れ
        max_wait=300,                           # メモリ作成の最大待機時間（5分）
        poll_interval=10                        # 10秒ごとにステータスをチェック
    )

    # メモリIDを抽出して出力
    memory_id = memory['id']
    logger.info(f"メモリが正常に作成されました。ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # メモリが既に存在する場合、IDを取得
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"メモリが既に存在します。既存のメモリIDを使用: {memory_id}")
except Exception as e:
    # メモリ作成中のエラーを処理
    logger.info(f"❌ エラー: {e}")
    import traceback
    traceback.print_exc()
    # エラー時のクリーンアップ - 部分的に作成されたメモリを削除
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"メモリをクリーンアップしました: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"メモリのクリーンアップに失敗: {cleanup_error}")

## ステップ3: LangGraphエージェント作成
LangGraphでエージェントを作成するために必要なライブラリをすべてインポートしましょう。

In [ ]:
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_aws import ChatBedrock

### LangGraphエージェントの実装

メモリツールを組み込んだLangGraphエージェントを作成しましょう：

In [ ]:
def create_agent(client, memory_id, actor_id, session_id):
    """LangGraphエージェントを作成・設定"""
    
    # LLMを初期化（必要に応じてモデルとパラメータを調整）
    llm = ChatBedrock(
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # または希望のモデル
        model_kwargs={"temperature": 0.1}
    )
    
    @tool
    def list_events():
        """必要に応じて最近の情報を検索するために使用されるツール""" 
        events = client.list_events(
                memory_id=memory_id,
                actor_id=actor_id,
                session_id=session_id,
                max_results=10
            )
        return events
        
    
    # ツールをLLMにバインド
    tools = [list_events]
    llm_with_tools = llm.bind_tools(tools)
    
    # システムメッセージ
    system_message = """あなたはパーソナルフィットネスコーチ、洗練されたフィットネスガイダンスアシスタントです。
                        目的：
                        - ユーザーのフィットネス目標に基づいてワークアウトルーチンを開発する支援
                        - ユーザーの運動の好み、制限、進歩を記憶する
                        - パーソナライズされたフィットネス推奨事項とトレーニングプランを提供
                        メモリ機能：
                        - list_eventsツールで最近のイベントにアクセスできます
                        """
    
    # チャットボットノードを定義
    def chatbot(state: MessagesState):
        raw_messages = state["messages"]
    
        # 重複や誤った配置を避けるために、既存のシステムメッセージを削除
        non_system_messages = [msg for msg in raw_messages if not isinstance(msg, SystemMessage)]
    
        # SystemMessageが常に最初になることを保証
        messages = [SystemMessage(content=system_message)] + non_system_messages
    
        latest_user_message = next((msg.content for msg in reversed(messages) if isinstance(msg, HumanMessage)), None)
    
        # ツールがバインドされたモデルからレスポンスを取得
        response = llm_with_tools.invoke(messages)
    
        # 該当する場合は会話を保存
        if latest_user_message and response.content.strip():  # レスポンスにコンテンツがあることを確認
            conversation = [
                (latest_user_message, "USER"),
                (response.content, "ASSISTANT")
            ]
            
            # すべてのメッセージテキストが空でないことを検証
            if all(msg[0].strip() for msg in conversation):  # 空のメッセージがないことを確認
                try:
                    client.create_event(
                        memory_id=memory_id,
                        actor_id=actor_id,
                        session_id=session_id,
                        messages=conversation
                    )
                except Exception as e:
                    print(f"会話の保存エラー: {str(e)}")
        
        # 完全なメッセージ履歴にレスポンスを追加
        return {"messages": raw_messages + [response]}
    
    # グラフを作成
    graph_builder = StateGraph(MessagesState)
    
    # ノードを追加
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # エッジを追加
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # エントリポイントを設定
    graph_builder.set_entry_point("chatbot")
    
    # グラフをコンパイル
    return graph_builder.compile()

### エージェント実行用ラッパーの作成

エージェントを実行するためのシンプルなラッパーを作成しましょう：

In [ ]:
def langgraph_bedrock(payload, agent):
    """
    ペイロードでエージェントを実行
    """
    user_input = payload.get("prompt")
    
    # LangGraphが期待する形式で入力を作成
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # 最終メッセージのコンテンツを抽出
    return response["messages"][-1].content

## ステップ4: LangGraphエージェントの実行
AgentCoreメモリ統合によりエージェントを実行できます。

In [ ]:
# この会話用にユニークなアクターIDとセッションIDを作成
actor_id = f"user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"workout-{datetime.now().strftime('%Y%m%d%H%M%S')}"

In [ ]:
# AgentCoreメモリ統合でエージェントを作成
agent = create_agent(client, memory_id, actor_id, session_id)

#### おめでとうございます！エージェントが準備できました！！

### エージェントをテストしてみましょう

メモリ機能をテストするためにエージェントとやり取りしてみましょう：

In [ ]:
response = langgraph_bedrock({"prompt": "こんにちは！今日が初日です。ワークアウトルーチンが必要です。"}, agent)
print(f"エージェント: {response}\n")

In [ ]:
response = langgraph_bedrock({"prompt": "筋肉を付けたいのですが、上腕二頭筋のルーチンを探しています。腰痛の問題があります。"}, agent)
print(f"エージェント: {response}\n")

In [ ]:
response = langgraph_bedrock({"prompt": "3つの運動と回数を教えてもらえますか？"}, agent)
print(f"エージェント: {response}\n")

### メモリ持続性のテスト

AgentCoreメモリ統合の力を真に実演するため、新しいエージェントインスタンスを作成して、以前の会話を思い出せるかテストしてみましょう：

In [ ]:
# 新しいエージェントインスタンスを作成（新しいセッションをシミュレート）
new_agent = create_agent(client, memory_id, actor_id, session_id)

# 新しいエージェントが設定を覚えているかテスト
response = langgraph_bedrock({
    "prompt": "また会いましたね！前回のワークアウトセッションについて思い出させてもらえますか？"
}, new_agent)

print("新しいエージェントセッション:\n")
print(f"エージェント: {response}")

## まとめ

このノートブックでは以下を実演しました：

1. AIエージェント用のAgentCoreメモリリソースの作成方法
2. メモリ統合を使用したLangGraphワークフローの構築
3. 会話履歴検索用のメモリツールの実装
4. 必要な時にメモリを知的に使用するエージェントの作成
5. エージェントインスタンス間でのメモリ持続性のテスト

この統合は、構造化ワークフロー（LangGraph）と堅牢なメモリシステム（AgentCoreメモリ）を組み合わせて、より知的でコンテキストを認識するAIエージェントを作成する力を示しています。

ここで実演したアプローチは、マルチエージェントシステム、抽出戦略を持つ長期記憶、会話のコンテキストに基づく専門的なメモリ検索など、より複雑な用途に拡張できます。

## クリーンアップ
このノートブックで使用したリソースをクリーンアップするためにメモリを削除しましょう。

In [ ]:
#client.delete_memory_and_wait(memory_id = memory_id, max_wait = 300, poll_interval =10)